## Problem

**Paula**, **Sue** and **Daisy** are mathematical geniuses. They can do any number of calculations in their head instantly. One day, they're given the task of figuring out two numbers. Those two numbers are in the range between $1$ and $1000$ (inclusive) and they're whole numbers (no decimal numbers). Both numbers can be the same too.

Paula is told the product of both numbers (one number multiplied with the other), Sue is told the sum of both numbers and Daisy is told the difference between the two numbers (one number substracted from the other). However, they're not allowed to tell those numbers to each other.

Then, the following conversation ensues:

> Paula: I don't know the two numbers.
>
> Sue: You don't need to tell me that, I've known that already.
>
> Paula: Ah, in that case, I know the numbers now.
>
> Sue: Ok, I know the numbers now too.
>
> Daisy: I still don't know the numbers. I can guess one number, which is likely to be one of the two numbers, but I don't know for sure.
>
> Paula: I know which number you're thinking of, but this one is wrong. It's not one of the two numbers.
>
> Daisy: Ah, then I know the numbers now too.

And that's it! 

Believe it or not, it's actually possible to figure out the two numbers. There's several different approaches to solving this, so I cannot give any hints, as one way may be easier for some people but harder for others, and vice versa.

Let $x$ and $y$ be the two numbers we want. To eliminate the symmetric solutions, we assume $x \leq y$.

Let $P = xy$, $S = x + y$, and $D = x - y$.

Since Paupla doesn't know the number, there should be at least 2 possible $(x,y)$ that can produce $P$. Let's find the set of possible products.

In [1]:
import collections

In [2]:
possible_products = set()
impossible_products = set()

temp_m = collections.defaultdict(int)
for x in range(1, 1001):
    for y in range(x, 1001):
        temp_m[x*y] += 1
        
for prod in temp_m.keys():
    if temp_m[prod] >= 2:
        possible_products.add(prod)
    if temp_m[prod] == 1:
        impossible_products.add(prod)

print(len(possible_products))
print(len(impossible_products))

103360
144723


Since Sue already knows from $S$ that Paupla doesn't know the two numbers, every combination of $x$ and $y$ that sums to $S$ cannot give a product that can uniquely tells Paula what $x$ and $y$ are. Based on this, we will search for the possible sums.

In [3]:
possible_sums = set()
impossible_sums = set()

for s in range(2, 2001):
    possible = True
    for x in range(1, s):
        y = s - x
        if x*y in impossible_products:
            possible = False
            break
    if possible:
        possible_sums.add(s)
    else:
        impossible_sums.add(s)

print(len(possible_sums))
print(len(impossible_sums))

235
1764


Since Paula can deduce $(x,y)$ from knowing that every combination of $x$ and $y$ that sums to $S$ cannot give a unique product, there must be only one possible $S$ where $P$ is possible and $P$ is not unique. Let's search for this $P$ and $S$.

In [4]:
p_s_possibility = set()
for x in range(1, 1001):
    for y in range(x, 1001):
        p_s_possibility.add((x*y, x+y))

In [5]:
nice_ps = []

for p in possible_products:
    count = 0
    nice_s = None
    
    for s in possible_sums:
        if (p, s) in p_s_possibility:
            count += 1
            nice_s = s
        
    if count == 1:
        nice_ps.append((p, nice_s))

print(len(nice_ps))

6984


Since Sue can deduce $(x,y)$ from knowing that there must be only one possible $S$ where $P$ is possible and $P$ is not unique, we must also have that there must be only one possible $P$ where $S$ is possible. Let's search for $P$ and $S$ again with this new constraint.

We first can reduce the set of possible products and sums.

In [6]:
possible_products = set()
possible_sums = set()
for p, s in nice_ps:
    possible_products.add(p)
    possible_sums.add(s)
    
print(len(possible_products))
print(len(possible_sums))

6984
215


In [7]:
nice_ps = []
for s in possible_sums:
    count = 0
    nice_p = None
    
    for p in possible_products:
        if (p, s) in p_s_possibility:
            count += 1
            nice_p = p
    
    if count == 1:
        nice_ps.append((nice_p, s))

print(len(nice_ps))

27


Now that we have reduced our search space to reasonable size, let's list the possible combinations of $(x,y)$.

In [8]:
nice_xys = set()
for (p, s) in nice_ps:
    xys = []
    for x in range(1, 1001):
        for y in range(x, 1001):
            if x*y == p and x+y == s:
                xys.append((x,y))
    
    nice_xys.update(xys)
    
    print(f"{xys} -> {(p, s)}")

[(1, 4)] -> (4, 5)
[(1, 8)] -> (8, 9)
[(1, 9)] -> (9, 10)
[(1, 15)] -> (15, 16)
[(11, 16)] -> (176, 27)
[(1, 32)] -> (32, 33)
[(8, 31)] -> (248, 39)
[(13, 32)] -> (416, 45)
[(16, 37)] -> (592, 53)
[(23, 32)] -> (736, 55)
[(16, 43)] -> (688, 59)
[(29, 32)] -> (928, 61)
[(32, 41)] -> (1312, 73)
[(16, 61)] -> (976, 77)
[(32, 53)] -> (1696, 85)
[(16, 73)] -> (1168, 89)
[(8, 89)] -> (712, 97)
[(37, 64)] -> (2368, 101)
[(32, 71)] -> (2272, 103)
[(43, 64)] -> (2752, 107)
[(8, 101)] -> (808, 109)
[(16, 97)] -> (1552, 113)
[(16, 103)] -> (1648, 119)
[(56, 71)] -> (3976, 127)
[(32, 101)] -> (3232, 133)
[(64, 73)] -> (4672, 137)
[(40, 109)] -> (4360, 149)


Since Daisy is also a genius, her search space should be a subset of what we have here. However, she also knows $D$, which is insufficient to help her pinpoint down the unique option. This means there are multiple options of $(x,y)$ giving the same $D$. Let's eliminate those that have unique $D$.

In [9]:
diff_count = collections.defaultdict(int)
nicer_xys = set()
nice_diffs = set()
for x, y in nice_xys:
    diff_count[x - y] += 1

for x, y in nice_xys:
    if diff_count[x-y] > 1:
        nice_diffs.add(x-y)
        nicer_xys.add((x,y))

for diff in nice_diffs:
    for x, y in nicer_xys:
        if x - y == diff:
            print(f"({x}, {y}) -> ({x*y}, {x+y}, {x-y})")

(37, 64) -> (2368, 101, -27)
(16, 43) -> (688, 59, -27)
(32, 53) -> (1696, 85, -21)
(43, 64) -> (2752, 107, -21)
(16, 37) -> (592, 53, -21)
(16, 97) -> (1552, 113, -81)
(8, 89) -> (712, 97, -81)
(23, 32) -> (736, 55, -9)
(64, 73) -> (4672, 137, -9)
(32, 41) -> (1312, 73, -9)
(32, 101) -> (3232, 133, -69)
(40, 109) -> (4360, 149, -69)
(1, 4) -> (4, 5, -3)
(29, 32) -> (928, 61, -3)


Paula should also be able to deduce the search space that Daisy can see. Since Paula knows the numbers, she knows exactly what options Daisy sees. Since Daisy claims that one number is likely to be one of the two numbers, this mean Daisy's guess has to appear in all but one option that has $D$.

By eyeballing, looking at $D = -9$. There are 2 options with one of the numbers being 32. None of the other $D$ has this property. This makes 32 seems like a more likely answer to Daisy. So when 32 is eliminated, the only option left is $(64,73)$, which should be our $(x,y)$.

Let's geocheck it :)

In [14]:
A,B,C,D=0,0,6,4
E,F,G,H=0,0,7,3

N = G*G*G + B*D + C*E*F + (C+A)*G + G
W = C*C*D*H + (B+D)*(E+G) + D - H

print(N,W) #????

392 461
